# King County Home Sales Analysis

* **Author:** Kevin Culver
* **Student pace:** self paced
* **Scheduled project review date/time:**  TBD
* **Instructor name:  Abhineet Kulkarni
* Blog post URL: TBD

In [2]:
#importing relevant modules
import pandas as pd
import numpy as np
import seaborn as sns



In [8]:
#loading dataset and column names
df = pd.read_csv('./data/kc_house_data.csv')
